In [8]:
import numpy as np
import cv2 as cv
import glob
from tqdm import tqdm

################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (3,3)
frameSize = (5312,2988)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 10
objp = objp * size_of_chessboard_squares_mm

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

folders = ["C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/1.1.2024-splitted videos",
"C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/2.1.2024-splitted videos",
"C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/31.12.2023-splitted videos"]
images = []
for folder in folders:
    images += glob.glob(folder + '/*.jpg')

img_counter = 0

for image in tqdm(images, desc="Processing images"):
    img = cv.imread(image)
    print("Processing image:", image)  # Add this line to print the name of the file
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Resize for faster processing
    scale_percent = 100  # percentage of original size
    width = int(gray.shape[1] * scale_percent / 100)
    height = int(gray.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_gray = cv.resize(gray, dim, interpolation=cv.INTER_AREA)

    # Find the chess board corners on resized image
    ret, corners = cv.findChessboardCorners(resized_gray, chessboardSize, None)

    if ret == True:
        img_counter += 1
        # Scale the corner coordinates back to original size
        corners = corners * (1/scale_percent*100)
        objpoints.append(objp)
        # Subpixel refinement on original image
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        #save the image with the corners drawn
        cv.imwrite('C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/images_with_corners/corners_found'+str(img_counter)+'.jpg', img)
        cv.namedWindow('img', cv.WINDOW_NORMAL)
        cv.imshow('img', img)
        cv.waitKey(3000)
        cv.destroyAllWindows()




Processing images:   0%|          | 0/832 [00:00<?, ?it/s]

Processing image: C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/1.1.2024-splitted videos\GX010080_0_39.jpg


Processing images:   0%|          | 1/832 [00:07<1:40:02,  7.22s/it]

Processing image: C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/1.1.2024-splitted videos\GX010080_10_948.jpg


Processing images:   0%|          | 2/832 [00:14<1:41:44,  7.35s/it]

...

Processing image: C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/1.1.2024-splitted videos\GX010080_22_1371.jpg


Processing images:   2%|▏         | 15/832 [01:54<1:39:49,  7.33s/it]

Processing image: C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/1.1.2024-splitted videos\GX010080_23_1377.jpg


Processing images:   2%|▏         | 15/832 [02:01<1:50:24,  8.11s/it]


KeyboardInterrupt: 

In [5]:
# Assuming frameSize is defined elsewhere
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Camera Matrix:\n")
print(cameraMatrix)
print("\n")
print("Distortion Coefficients:\n")
print(dist)
print("\n")
print("Rotation Vectors:\n")
print(rvecs)
print("\n")
print("Translation Vectors:\n")
print(tvecs)
print("\n")
print("Number of images used for calibration: ", img_counter)
#save the camera matrix and distortion coefficients to a file

Camera Matrix:

[[9.63725147e+02 0.00000000e+00 2.49116761e+03]
 [0.00000000e+00 7.93390202e+02 1.65230204e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


Distortion Coefficients:

[[ 0.02783515 -0.00103007 -0.0065827  -0.03802787 -0.00256279]]


Rotation Vectors:

(array([[-0.01336861],
       [-0.84544614],
       [-1.09306093]]), array([[ 0.66250208],
       [-0.61255868],
       [ 0.76073061]]), array([[-0.90302668],
       [ 0.36220876],
       [ 1.23064732]]))


Translation Vectors:

(array([[ 23.73958173],
       [ 19.10378506],
       [120.60663514]]), array([[ 50.90136861],
       [-64.47876293],
       [ 79.78769095]]), array([[ 91.35305713],
       [100.27524436],
       [ 87.8421484 ]]))


Number of images used for calibration:  3


In [6]:
ret

22.83484698388246

In [ ]:
np.savez("src/measurementscalibration.npz", cameraMatrix=cameraMatrix, dist=dist, rvecs=rvecs, tvecs=tvecs,ret=ret,img_counter=img_counter)

In [ ]:
#save all to csv file
import csv

with open('src/measurementscalibration.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(["Camera Matrix"])
    writer.writerow(cameraMatrix)
    writer.writerow(["Distortion Coefficients"])
    writer.writerow(dist)
    writer.writerow(["Rotation Vectors"])
    writer.writerow(rvecs)
    writer.writerow(["Translation Vectors"])
    writer.writerow(tvecs)
    writer.writerow(["Number of images used for calibration: ", img_counter])
    #projection error
    writer.writerow(["Projection Error"])
    writer.writerow(ret)

print("Calibration done. Files saved to src folder")




In [ ]:






############## CALIBRATION #######################################################




############## UNDISTORTION #####################################################

img = cv.imread('C:\Users\gbo10\OneDrive\research\italy\torino/computer vision/test/images/GX010097_MP4-29_jpg.rf.4f866c5d8b12dbf51b8e1307c87e4447.jpg')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)



# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)




# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print("total error: {}".format(mean_error/len(objpoints)))

import matplotlib.pyplot as plt

# Plotting the reprojection error
plt.plot(range(len(objpoints)), [cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2) for i in range(len(objpoints))])
plt.xlabel('Image Index')
plt.ylabel('Reprojection Error')
plt.title('Reprojection Error for Camera Calibration')
plt.show()

